In [ ]:
class Instruction:

    def __init__(self, op, arg1=None, arg2=None, result=None):

        self.op = op
        self.arg1 = arg1
        self.arg2 = arg2
        self.result = result

    def get_used_variables(self):

        used_variables = set()
        if self.arg1 is not None:
            used_variables.add(self.arg1)
        if self.arg2 is not None:
            used_variables.add(self.arg2)
        return used_variables

    def get_defined_variables(self):

        defined_variables = set()
        if self.result is not None:
            defined_variables.add(self.result)
        return defined_variables


def allocate_registers_top_down(instructions, num_registers):
    available_registers = set(range(num_registers))

    register_allocations = {}

    for instruction in instructions:

        used_variables = instruction.get_used_variables()

        for variable in used_variables:

            if variable not in register_allocations:

                if not available_registers:
                    spilled_variable = min(register_allocations, key=register_allocations.get)
                    available_registers.add(register_allocations[spilled_variable])
                    del register_allocations[spilled_variable]

                register = available_registers.pop()
                register_allocations[variable] = register

        defined_variables = instruction.get_defined_variables()

        for variable in defined_variables:

            if variable in register_allocations:
                available_registers.add(register_allocations[variable])
                del register_allocations[variable]

    return register_allocations

def allocate_registers_bottom_up(instructions, num_registers):

    available_registers = set(range(num_registers))

    register_allocations = {}

    for instruction in reversed(instructions):

        defined_variables = instruction.get_defined_variables()

        for variable in defined_variables:

            if variable in register_allocations:
                available_registers.add(register_allocations[variable])
                del register_allocations[variable]

        used_variables = instruction.get_used_variables()

        for variable in used_variables:

            if variable not in register_allocations:

                if not available_registers:
                    spilled_variable = min(register_allocations, key=register_allocations.get)
                    available_registers.add(register_allocations[spilled_variable])
                    del register_allocations[spilled_variable]

                register = available_registers.pop()
                register_allocations[variable] = register

    return register_allocations

def main():
    instructions = [
        Instruction("ADD", "a", "b", "c"),  # a = b + c
        Instruction("MUL", "d", "a", "e"),  # d = a * e
        Instruction("SUB", "f", "d", "b"),  # f = d - b
        Instruction("DIV", "g", "f", "c"),  # g = f / c
        Instruction("ADD", "h", "g", "a")   # h = g + a
    ]

    num_registers = 8

    register_allocations_top_down = allocate_registers_top_down(instructions, num_registers)

    print("Register allocations (top-down):")
    for variable, register in register_allocations_top_down.items():
        print(f"{variable}: R{register}")

    register_allocations_bottom_up = allocate_registers_bottom_up(instructions, num_registers)

    if register_allocations_bottom_up is not None:
      print("Register allocations (bottom-up):")
      for variable, register in register_allocations_bottom_up.items():
        print(f"{variable}: R{register}")

if __name__=="__main__":
    main()

Register allocations (top-down):
d: R2
f: R3
g: R4
h: R5
Register allocations (bottom-up):
h: R0
g: R1
f: R2
d: R3
a: R4
b: R5
